# Lab session of 23/03/2022

## Graph problem 3: The Traveling Salesperson Problem (TSP)

Find the shortest Hamiltonian circuit through $n$ nodes where the distance between any two nodes is known. In particular:

1. Generate $k$ points by creating random $x,y$ coordinates for each; the coordinates should be in the interval $[0,100]$;
2. Assume the graph $G = (V,A)$ on which to solve the TSP is _complete_, i.e. any two pairs are connected by an arc;
3. Create the optimization model for the TSP by only adding _flow conservation_ constraints, i.e., do not add any subtour elimination constraints;
4. Iteratively solve the optimization model and then add subtour elimination constraints until the solution is a single (optimal) tour.

## Solution

Similar to problem 1 (shortest path tree), we'll perform a step at a time and visualize what we obtain in order to provide some intuition for each step. We'll use similar code for creating coordinates.

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

import numpy as np
import math

k = 7  # number of nodes

grid_size = 100 # size of the grid of points

# Create k random points with two coordinates. Multiplying by grid_size yields
# random numbers between 0 and 100.
point = grid_size * np.random.random((k,2))

# Define the set of vertices of the graph as the list of numbers from 0 to k-1

# Determine the distance between each point


Let's plot these points on the grid. We use `matplotlib` and specifically the function `matplotlib.pyplot`. We then create a complete graph and draw it on the grid.

In [ ]:
import matplotlib.pyplot as plot
plot.scatter(point[:,0], point[:,1])
plot.show()

In [ ]:
import networkx as nx

# Set of edges: note the condition that i<j as these are edges, not arcs,
# i.e. they are not directed.
E = [(i,j) for i in V for j in V if i < j]

# Define a dictionary whose keys are the nodes, and the values are tuples
# containing the (x,y) coordinates of each point
coord = {i: (point[i,0], point[i,1]) for i in V}

g = nx.Graph()

g.add_nodes_from(V)
g.add_edges_from(E)

nx.draw(g, pos=coord)

We have all data we need and proceed to create a MIP model for the TSP. Let's write the complete one, with the exponentially large set of subtour-elimination constraints:

$$
\begin{array}{lll}
   \min & \sum_{(i,j) \in E} d_{ij} x_{ij}\\
   \textrm{s.t.} & \sum_{j \in V: (i,j) \in E} x_{ij} = 2 & \forall i\in V\\
                 & \sum_{(i,j) \in E: i\in C, j\in C} x_{ij} \le |C| - 1 & \forall C\subset V: C \neq \emptyset\\
                 & x_{ij} \in \{0,1\} & \forall (i,j) \in E
\end{array}
$$

In the first version of our model, we won't include any subtour-elimination constraint but will only add the first set.

In [ ]:
# Create model, add variables

The flow conservation constraints requires that the incoming flow equal the outgoing flow. Instead of adding these constraints in a loop, we add them using a generator.

In [ ]:
# Add conservation constraints

The objective function is the same as for the previous problem: the sum over all arcs $(i,j)$ of their length $d_{ij}$ (i.e. the distance between nodes $i$ and $j$) multiplied by the variable $x_{ij}$.

In [1]:
# Add objective function and solve problem

We can rewrite the function for drawing the MIP solution, which might contain subtours.

In [ ]:
def draw_solution(V, A, x):
    g = nx.Graph()

    # Draw the whole graph first: all nodes, all arcs, no highlighting
    g.add_nodes_from(V)
    g.add_edges_from([(i,j) for (i,j) in A])
    nx.draw(g, pos=coord, with_labels=True, node_color="white")

    # Reset the graph and add only the arcs that belong to the solution, 
    # i.e. those for which the optimal value of the variable x[i,j] is nonzero
    g.clear()
    g.add_edges_from([(i,j) for (i,j) in A if x[i,j].x > 0.5])
    nx.draw(g, pos=coord, width=4, edge_color='red', with_labels=True, node_color='white')

    # finally, draw a graph consisting of the sole root node, highlighted in green
    g.clear()
    g.add_node(0)
    nx.draw(g, pos={0: coord[0]}, node_color='white', with_labels=True)

    
# after defining the function, call it with the current data
draw_solution (V, E, x)

Did we get a Hamiltonian circuit or subtours? In the latter case, we must separate subtour elimination constraints. We do it by hand for the first few iterations.


In [ ]:
# Prototype:
#
# m.add_constr(x[0,4] + x[4,6] + x[0,5] + x[5,6] <= 3)
# m.optimize()
# draw_solution (V, A, x)

How do we make this automatic? I.e. how do we repeat this step by adding the appropriate subtour-elimination constraint until we find a Hamiltonian circuit?